In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary
import torch.nn.functional as F
import time
import numpy as np

## 读取数据

## 定义模型

In [16]:
class LeNetNModel(torch.nn.Module):
    def __init__(self):
        super(LeNetNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=3)
        self.subsampling1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.subsampling2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5)
        self.linear1 = nn.Linear(120, 84)
        self.linear2 = nn.Linear(84, 10)
    def forward(self, x):
        #x1 = F.sigmoid(self.conv1(x))
        x1 = F.relu(self.conv1(x))
        x2 = self.subsampling1(x1)
        #x3 = F.sigmoid(self.conv2(x2))
        x3 = F.relu(self.conv2(x2))
        x4 = self.subsampling2(x3)
        #x5 = F.sigmoid(self.conv3(x4))
        x5 = F.relu(self.conv3(x4))
        x6 = self.linear1(x5.view(x5.shape[0], -1))
        y = self.linear2(x6)
        return y
    

## 训练模型

In [17]:
def training_loop(net, optimizer, loss, n_epochs, train_set, test_set, batch_size=64):
    train_iter = torch.utils.data.DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
    test_iter = torch.utils.data.DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)
    net = net.to('cuda')
    for epoch in range(n_epochs):
        train_loss_sum , batch_count, start = 0, 0, time.time()
        for X, y in train_iter:
            X = X.to('cuda')
            y = y.to('cuda')
            y_pre = net(X)
            l = loss(y_pre, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_loss_sum += l.item()
            batch_count += 1
        print("epochs: %d, loss: %2f, time: %2f sec" % (epoch+1, train_loss_sum / batch_count, 
                                                       (time.time() - start)))
        

In [19]:
if __name__ == "__main__":
    savePath = '../Chapter3/Fashion-MNIST/'
    mnist_train = torchvision.datasets.FashionMNIST(root=savePath, train=True, download=True, transform=transforms.ToTensor())
    mnist_test = torchvision.datasets.FashionMNIST(root=savePath, train=False, download=True, transform=transforms.ToTensor())
    
    lr = 0.001
    batch_size = 128
    n_epochs = 5
    net = LeNetNModel()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    loss = torch.nn.CrossEntropyLoss()
    print(net)
    #summary(net.to('cuda'), (1, 28, 28))
    training_loop(net=net, optimizer=optimizer, loss=loss, n_epochs=n_epochs, train_set=mnist_train,
                  test_set=mnist_test, batch_size=batch_size)
    

LeNetNModel(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(3, 3))
  (subsampling1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (subsampling2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (linear1): Linear(in_features=120, out_features=84, bias=True)
  (linear2): Linear(in_features=84, out_features=10, bias=True)
)
epochs: 1, loss: 0.693027, time: 22.115390 sec
epochs: 2, loss: 0.428100, time: 22.668360 sec
epochs: 3, loss: 0.362321, time: 23.482848 sec
epochs: 4, loss: 0.325018, time: 23.634640 sec
epochs: 5, loss: 0.302060, time: 23.663747 sec


In [ ]:
class Lenet(nn.Module):
    def __init__(self):
        super(Lenet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 6, 3, stride=1, padding=1),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 5, stride=1, padding=0),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 120, 5)
        )

        self.fc = nn.Sequential(
            #nn.Linear(400, 120),
            nn.Linear(120, 84),
            nn.Linear(84, 10)
        )


    def forward(self, x):
        out = self.conv(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
net2 = Lenet()
summary(net2.to('cuda'), (1, 28, 28))